# Generate a Modified Nested Set Index from NGA TDX-Hydro

This notebook demonstrates how to use functions in the [WikiWatershed/global-hydrography](https://github.com/WikiWatershed/global-hydrography) package to generate a modified nested set index using the TDX-Hydro datasets released by the [US National Geospatial-Intelligence Agency (NGA)](https://www.nga.mil).

This example notebook assumes that you have already downloaded the applicable data using the example provided in the `1_GetData.ipynb` notebook. This notebook also assumes that you will have completed the necessary setup steps outline in the **[Installation Instructions](README.md#get-started)** (and also completed as part of the notebook `1_GetData.ipynb`) 

# Python Imports

In this step we will import the necessary python dependencies for this example

In [64]:
from pathlib import Path
import re
from importlib import reload

import pyogrio
import geopandas as gpd
import pandas as pd

import global_hydrography as gh
from global_hydrography.delineation.mnsi import modified_nest_set_index
from global_hydrography.preprocess import TDXPreprocessor

In [2]:
# Explore the namespace for global-hydrography modules, functions, etc.
dir(gh)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'delineation',
 'io',
 'mnsi',
 'preprocess',
 'process']

# Compile files that need to be processed

In this step we will compile a list of the files that need to be processed to have a modified nested set index. Note this step assumes that you have downloaded the files to the same directory and used the same naming convention as the `1_GetData.ipynb` example notebook. If you have opted to use a different location or naming convention you will need to modify this step accordingly.

In [3]:
# Confirm your current working directory (cwd) and repo/project directory
working_dir = Path.cwd()
project_dir = working_dir.parent
data_dir = project_dir / 'data_temp' # a temporary data directory that we .gitignore
tdx_dir = data_dir / 'nga'

In [4]:
#Scan the files in the data directory and only pull of the streamnet (blueline) files
files_to_process = []
for item in tdx_dir.iterdir():
    if item.is_file() and 'streamnet' in item.name:
        files_to_process.append(item)

In [5]:
files_to_process

[PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_1020011530_01.gpkg'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.parquet'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.gpkg-shm'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.gpkg-wal'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01.gpkg'),
 PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01_mnsi.parquet')]

# Explore Workflows and Test Functions

## Read Files

In [6]:
file = files_to_process[4]
file.name

'TDX_streamnet_7020038340_01.gpkg'

In [7]:
file.suffix

'.gpkg'

In [8]:
#parse the file name to get the HDX Basin Id
tdx_basin_id = int(re.search("\d{10}",file.name).group(0))
tdx_basin_id

7020038340

In [9]:
info = pyogrio.read_info(file, layer=0)
info

{'layer_name': 'TDX_streamnet_7020038340_01',
 'crs': 'EPSG:4326',
 'encoding': 'UTF-8',
 'fields': array(['LINKNO', 'DSLINKNO', 'USLINKNO1', 'USLINKNO2', 'DSNODEID',
        'strmOrder', 'Length', 'Magnitude', 'DSContArea', 'strmDrop',
        'Slope', 'StraightL', 'USContArea', 'WSNO', 'DOUTEND', 'DOUTSTART',
        'DOUTMID'], dtype=object),
 'dtypes': array(['int32', 'int32', 'int32', 'int32', 'int64', 'int32', 'float64',
        'int32', 'float64', 'float64', 'float64', 'float64', 'float64',
        'int32', 'float64', 'float64', 'float64'], dtype=object),
 'fid_column': 'fid',
 'geometry_name': 'geom',
 'geometry_type': 'LineString',
 'features': 140097,
 'total_bounds': (-89.8212222222222,
  24.5589999999989,
  -66.1413333333321,
  46.4454444444444),
 'driver': 'GPKG',
 'capabilities': {'random_read': True,
  'fast_set_next_by_index': True,
  'fast_spatial_filter': True,
  'fast_feature_count': True,
  'fast_total_bounds': True},
 'layer_metadata': {'DBF_DATE_LAST_UPDATE': '202

In [10]:
info['layer_name']

'TDX_streamnet_7020038340_01'

In [11]:
int(re.search("\d{10}",info['layer_name']).group(0))

7020038340

In [12]:
info['layer_metadata']

{'DBF_DATE_LAST_UPDATE': '2021-12-08'}

In [13]:
#open the file as GeoDataFrame, using the fastest direct method from issue #1
gdf = gpd.read_file(file, engine='pyogrio', layer=0, use_arrow=True)
gdf.info()
gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   LINKNO      140097 non-null  int32   
 1   DSLINKNO    140097 non-null  int32   
 2   USLINKNO1   140097 non-null  int32   
 3   USLINKNO2   140097 non-null  int32   
 4   DSNODEID    140097 non-null  int64   
 5   strmOrder   140097 non-null  int32   
 6   Length      140097 non-null  float64 
 7   Magnitude   140097 non-null  int32   
 8   DSContArea  140097 non-null  float64 
 9   strmDrop    140097 non-null  float64 
 10  Slope       140097 non-null  float64 
 11  StraightL   140097 non-null  float64 
 12  USContArea  140097 non-null  float64 
 13  WSNO        140097 non-null  int32   
 14  DOUTEND     140097 non-null  float64 
 15  DOUTSTART   140097 non-null  float64 
 16  DOUTMID     140097 non-null  float64 
 17  geometry    140097 non-null  geometry
dtypes: float64(9), g

,LINKNO,DSLINKNO,USLINKNO1,USLINKNO2,DSNODEID,strmOrder,Length,Magnitude,DSContArea,strmDrop,Slope,StraightL,USContArea,WSNO,DOUTEND,DOUTSTART,DOUTMID,geometry
0,0,1777,-1,-1,-1,1,3847.9,1,9567845.0,42.07,0.010933,3233.7,5254867.5,0,45853.6,49701.4,47777.5,"LINESTRING (-69.67822 46.41356, -69.67822 46.4..."
1,1,2369,-1,-1,-1,1,2251.3,1,8768556.0,34.66,0.015397,1749.2,4320561.0,1,44802.7,47054.1,45928.4,"LINESTRING (-69.68589 46.40778, -69.686 46.407..."
2,593,1777,-1,-1,-1,1,1469.3,1,8466694.0,11.98,0.008153,1286.2,4319318.0,593,45853.6,47322.9,46588.3,"LINESTRING (-69.67822 46.41356, -69.67811 46.4..."
3,1777,2369,0,593,-1,2,1050.9,2,19939082.0,0.91,0.000870,871.8,18034788.0,1777,44802.7,45853.6,45328.2,"LINESTRING (-69.68589 46.40778, -69.68589 46.4..."
4,2,4146,-1,-1,-1,1,3551.0,1,9120895.0,67.48,0.019002,2593.6,5267176.0,2,41041.1,44591.7,42816.4,"LINESTRING (-69.687 46.37911, -69.687 46.379, ..."


In [14]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Preprocess GeoDataFrame

In [15]:
# Create an instance of the preprocessor class from our package
preprocessor = TDXPreprocessor()

In [16]:
test0_gdf = gdf.copy()

In [17]:
%%timeit
# apply preprocessing to make linkno globally unique
preprocessor.tdx_to_global_linkno(test0_gdf, tdx_basin_id)

5.72 ms ± 3.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# apply preprocessing to make linkno globally unique
processed_gdf = preprocessor.tdx_to_global_linkno(gdf, tdx_basin_id)
processed_gdf.info()
processed_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   LINKNO      140097 non-null  int32   
 1   DSLINKNO    140097 non-null  int32   
 2   USLINKNO1   140097 non-null  int32   
 3   USLINKNO2   140097 non-null  int32   
 4   DSNODEID    140097 non-null  int64   
 5   strmOrder   140097 non-null  int32   
 6   Length      140097 non-null  float64 
 7   Magnitude   140097 non-null  int32   
 8   DSContArea  140097 non-null  float64 
 9   strmDrop    140097 non-null  float64 
 10  Slope       140097 non-null  float64 
 11  StraightL   140097 non-null  float64 
 12  USContArea  140097 non-null  float64 
 13  WSNO        140097 non-null  int32   
 14  DOUTEND     140097 non-null  float64 
 15  DOUTSTART   140097 non-null  float64 
 16  DOUTMID     140097 non-null  float64 
 17  geometry    140097 non-null  geometry
dtypes: float64(9), g

,LINKNO,DSLINKNO,USLINKNO1,USLINKNO2,DSNODEID,strmOrder,Length,Magnitude,DSContArea,strmDrop,Slope,StraightL,USContArea,WSNO,DOUTEND,DOUTSTART,DOUTMID,geometry
0,750000000,750001777,-1,-1,-1,1,3847.9,1,9567845.0,42.07,0.010933,3233.7,5254867.5,0,45853.6,49701.4,47777.5,"LINESTRING (-69.67822 46.41356, -69.67822 46.4..."
1,750000001,750002369,-1,-1,-1,1,2251.3,1,8768556.0,34.66,0.015397,1749.2,4320561.0,1,44802.7,47054.1,45928.4,"LINESTRING (-69.68589 46.40778, -69.686 46.407..."
2,750000593,750001777,-1,-1,-1,1,1469.3,1,8466694.0,11.98,0.008153,1286.2,4319318.0,593,45853.6,47322.9,46588.3,"LINESTRING (-69.67822 46.41356, -69.67811 46.4..."
3,750001777,750002369,750000000,750000593,-1,2,1050.9,2,19939082.0,0.91,0.000870,871.8,18034788.0,1777,44802.7,45853.6,45328.2,"LINESTRING (-69.68589 46.40778, -69.68589 46.4..."
4,750000002,750004146,-1,-1,-1,1,3551.0,1,9120895.0,67.48,0.019002,2593.6,5267176.0,2,41041.1,44591.7,42816.4,"LINESTRING (-69.687 46.37911, -69.687 46.379, ..."


## Drop Useless Columns

In [19]:
preprocessor.tdx_drop_useless_columns(processed_gdf)

In [20]:
processed_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   LINKNO      140097 non-null  int32   
 1   DSLINKNO    140097 non-null  int32   
 2   USLINKNO1   140097 non-null  int32   
 3   USLINKNO2   140097 non-null  int32   
 4   strmOrder   140097 non-null  int32   
 5   Length      140097 non-null  float64 
 6   Magnitude   140097 non-null  int32   
 7   DSContArea  140097 non-null  float64 
 8   strmDrop    140097 non-null  float64 
 9   Slope       140097 non-null  float64 
 10  StraightL   140097 non-null  float64 
 11  USContArea  140097 non-null  float64 
 12  DOUTEND     140097 non-null  float64 
 13  DOUTSTART   140097 non-null  float64 
 14  DOUTMID     140097 non-null  float64 
 15  geometry    140097 non-null  geometry
dtypes: float64(9), geometry(1), int32(6)
memory usage: 13.9 MB


In [21]:
# Drop columns with no value. See `sandbox/explore_data_sources.ipynb`
useless_columns = [
    'WSNO', # identical values to 'LINKNO'
    'DSNODEID', # all -1
]
columns_to_drop = []
for column in useless_columns:
    if any(test0_gdf.columns.isin([column])):
        columns_to_drop.append(column)

columns_to_drop

['WSNO', 'DSNODEID']

In [22]:
# Produces error if already dropped
# processed_gdf.drop(columns=columns_to_drop, inplace=True)

In [23]:
# Test function to avoid missing columns error
preprocessor.tdx_drop_useless_columns(processed_gdf)

## Compute Modified Nested Set Index

In [24]:
test_gdf = processed_gdf.copy(deep=True)

In [25]:
# %%timeit
# compute the modified nested set index is quite fast!! 
# modified_nest_set_index(test_gdf)
# 6.17 s ± 73.9 ms

In [26]:
#compute the modified nested set index
mnsi_gdf = modified_nest_set_index(processed_gdf)
mnsi_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 140097 entries, 0 to 140096
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   LINKNO         140097 non-null  int32   
 1   DSLINKNO       140097 non-null  int32   
 2   USLINKNO1      140097 non-null  int32   
 3   USLINKNO2      140097 non-null  int32   
 4   ROOT_ID        140097 non-null  int32   
 5   DISCOVER_TIME  140097 non-null  int32   
 6   FINISH_TIME    140097 non-null  int32   
 7   strmOrder      140097 non-null  int32   
 8   Length         140097 non-null  float64 
 9   Magnitude      140097 non-null  int32   
 10  DSContArea     140097 non-null  float64 
 11  strmDrop       140097 non-null  float64 
 12  Slope          140097 non-null  float64 
 13  StraightL      140097 non-null  float64 
 14  USContArea     140097 non-null  float64 
 15  DOUTEND        140097 non-null  float64 
 16  DOUTSTART      140097 non-null  float64 
 17  DO

In [27]:
mnsi_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [28]:
mnsi_gdf.index

RangeIndex(start=0, stop=140097, step=1)

In [51]:
# Set 'LINKNO' as index, to facilitate selection
# and interoperability with basins
mnsi_gdf.set_index('LINKNO', inplace=True)

In [53]:
mnsi_gdf[['DSLINKNO', 'USLINKNO1', 'USLINKNO2', 
          'ROOT_ID', 'DISCOVER_TIME', 'FINISH_TIME' ]]

,DSLINKNO,USLINKNO1,USLINKNO2,ROOT_ID,DISCOVER_TIME,FINISH_TIME
LINKNO,,,,,,
750000000,750001777,-1,-1,750021317,52,53
750000001,750002369,-1,-1,750021317,49,50
750000593,750001777,-1,-1,750021317,51,52
750001777,750002369,750000000,750000593,750021317,50,53
750000002,750004146,-1,-1,750021317,47,48
...,...,...,...,...,...,...
750000587,-1,-1,-1,750000587,1,2
750001180,-1,-1,-1,750001180,1,2
750001772,-1,-1,-1,750001772,1,2


In [54]:
mnsi_gdf.ROOT_ID.value_counts().sort_values()

ROOT_ID
750005168        1
750000464        1
750075251        1
750000462        1
750070517        1
             ...  
750170058     4979
750301090     5929
750093395     6453
750127466     9771
750236632    14095
Name: count, Length: 1635, dtype: int64

## Write GeoParquet File

In [58]:
tdx_parquet_path = tdx_dir / f"{info['layer_name']}_mnsi_test.parquet"
tdx_parquet_path


PosixPath('/Users/aaufdenkampe/Documents/Python/global-hydrography/data_temp/nga/TDX_streamnet_7020038340_01_mnsi_test.parquet')

In [56]:
# Test to see if adding metadata to gdf gets saved to parquet
mnsi_gdf.source_info = info['layer_metadata']
mnsi_gdf.source_info

{'DBF_DATE_LAST_UPDATE': '2021-12-08'}

In [33]:
#write back to a file
mnsi_gdf.to_parquet(tdx_parquet_path, compression='zstd')

In [34]:
pyogrio.read_info(tdx_parquet_path)

{'layer_name': 'TDX_streamnet_7020038340_01_mnsi',
 'crs': 'EPSG:4326',
 'encoding': 'UTF-8',
 'fields': array(['LINKNO', 'DSLINKNO', 'USLINKNO1', 'USLINKNO2', 'ROOT_ID',
        'DISCOVER_TIME', 'FINISH_TIME', 'strmOrder', 'Length', 'Magnitude',
        'DSContArea', 'strmDrop', 'Slope', 'StraightL', 'USContArea',
        'DOUTEND', 'DOUTSTART', 'DOUTMID'], dtype=object),
 'dtypes': array(['int32', 'int32', 'int32', 'int32', 'int32', 'int32', 'int32',
        'int32', 'float64', 'int32', 'float64', 'float64', 'float64',
        'float64', 'float64', 'float64', 'float64', 'float64'],
       dtype=object),
 'fid_column': '',
 'geometry_name': 'geometry',
 'geometry_type': 'LineString',
 'features': 140097,
 'total_bounds': (-89.82122222222222,
  24.558999999998896,
  -66.14133333333214,
  46.44544444444445),
 'driver': 'Parquet',
 'capabilities': {'random_read': False,
  'fast_set_next_by_index': True,
  'fast_spatial_filter': False,
  'fast_feature_count': True,
  'fast_total_bounds': 

# Alternate: Create Basins with MNSI

To only read the basin's file for delinating the upstream watershed boundary.

Using new functions in source directory to streamline production.

In [35]:
# Reload module after editing
reload(gh.process)

<module 'global_hydrography.process' from '/Users/aaufdenkampe/Documents/Python/global-hydrography/src/global_hydrography/process.py'>

## `select_tdx_files()` function

In [48]:
tdx_hydro_region = 7020038340

In [38]:
# Test new select_tdx_files()
streamnet_file, basins_file = gh.process.select_tdx_files(
    tdx_dir, tdxhydroregion,'.gpkg')

streamnet_file.name

'TDX_streamnet_7020038340_01.gpkg'

In [39]:
basins_file.name

'TDX_streamreach_basins_7020038340_01.gpkg'

## Process basins file

In [47]:
# open basins file as GeoDataFrame
basins_gdf = gpd.read_file(basins_file, engine='pyogrio', layer=0, use_arrow=True)

In [49]:
# Rename 'streamID' to 'LINKNO' to facilitate interoperability 
# with streamnet files
basins_gdf.rename(columns={'streamID':'LINKNO'}, inplace=True)

# apply preprocessing to make linkno globally unique
preprocessor.tdx_to_global_linkno(basins_gdf, tdx_hydro_region)

# Set 'LINKNO' as index, to facilitate selection
basins_gdf.set_index('LINKNO', inplace=True)

basins_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 140053 entries, 750000001 to 750327711
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  140053 non-null  geometry
dtypes: geometry(1)
memory usage: 2.1 MB


## Move MNSI fields from streamnet to basins

In [77]:
stream_mnsi_gdf = mnsi_gdf.copy(deep=True)

In [78]:
columns_to_merge = ['DSContArea', 'USContArea']

# Merge confirms that their LINKNO values match
# Although there are not as many basins as there are stream reaches!
basins_test_gdf = pd.merge(
    basins_test_gdf, 
    stream_mnsi_gdf[columns_to_merge], 
    how='right', 
    on='LINKNO',
)
basins_test_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 140097 entries, 750000000 to 750000589
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   geometry    140053 non-null  geometry
 1   DSContArea  140097 non-null  float64 
 2   USContArea  140097 non-null  float64 
dtypes: float64(2), geometry(1)
memory usage: 4.3 MB


In [81]:
# Explore stream links with no basin geometry.
streams_no_basins_gdf = stream_mnsi_gdf[basins_test_gdf.geometry==None]
streams_no_basins_gdf.info()
streams_no_basins_gdf.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 44 entries, 750000000 to 750020103
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   DSLINKNO       44 non-null     int32   
 1   USLINKNO1      44 non-null     int32   
 2   USLINKNO2      44 non-null     int32   
 3   ROOT_ID        44 non-null     int32   
 4   DISCOVER_TIME  44 non-null     int32   
 5   FINISH_TIME    44 non-null     int32   
 6   strmOrder      44 non-null     int32   
 7   Length         44 non-null     float64 
 8   Magnitude      44 non-null     int32   
 9   DSContArea     44 non-null     float64 
 10  strmDrop       44 non-null     float64 
 11  Slope          44 non-null     float64 
 12  StraightL      44 non-null     float64 
 13  USContArea     44 non-null     float64 
 14  DOUTEND        44 non-null     float64 
 15  DOUTSTART      44 non-null     float64 
 16  DOUTMID        44 non-null     float64 
 17  geometry       44 n

,DSLINKNO,USLINKNO1,USLINKNO2,ROOT_ID,DISCOVER_TIME,FINISH_TIME,strmOrder,Length,Magnitude,DSContArea,strmDrop,Slope,StraightL,USContArea,DOUTEND,DOUTSTART,DOUTMID,geometry
LINKNO,,,,,,,,,,,,,,,,,,
750000000,750001777,-1,-1,750021317,52,53,1,3847.9,1,9.567845e+06,42.07,0.010933,3233.7,5.254868e+06,45853.6,49701.4,47777.5,"LINESTRING (-69.67822 46.41356, -69.67822 46.4..."
750100709,750101301,750090644,750068149,750100710,5,16,3,0.0,6,9.573266e+07,0.00,0.000000,0.0,9.573266e+07,7889.2,7889.2,7889.2,"LINESTRING (-69.74322 43.89322, -69.74322 43.8..."
750155209,750155801,750154617,750113177,750170058,1767,4980,7,0.0,1607,2.410359e+10,0.00,0.000000,0.0,2.410359e+10,234019.3,234019.3,234019.3,"LINESTRING (-73.75744 42.54056, -73.75744 42.5..."
750127463,750128055,750141672,750010841,750129283,3471,4886,6,0.0,708,1.099047e+10,0.00,0.000000,0.0,1.099047e+10,423795.8,423795.8,423795.8,"LINESTRING (-78.23989 39.65089, -78.23989 39.6..."
750099079,750099671,750055269,750055861,750102638,89,96,2,0.0,4,8.714499e+07,0.00,0.000000,0.0,8.714499e+07,68889.3,68889.3,68889.3,"LINESTRING (-76.08133 38.47233, -76.08133 38.4..."


In [84]:
streams_no_basins_gdf.Length.value_counts()

Length
0.0       43
3847.9     1
Name: count, dtype: int64

**NOTE: All but one have zero stream length. The one with a lenghth is a headwater stream at the edge of the TDXHydroRegion.

In [83]:
# Write to GeoPackage to explore in QGIS
pyogrio.write_dataframe(streams_no_basins_gdf, tdx_dir / 'streams_no_basins.gpkg')
# looks like random mistakes


In [ ]:
DISCOVER = "DISCOVER_TIME"
FINISH = "FINISH_TIME"
ROOT = "ROOT_ID"
# at column locations right after other LINK info
for f in (FINISH, DISCOVER, ROOT):
    basins_gdf.insert(0, f, None)